## Import cell

In [30]:
from load_data import load_images
from losses import content_loss, style_loss
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from PIL import Image
from torch.autograd import Variable


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device('cuda:0')
print('Current Device:', device)

Current Device: cuda:0


## Preparation cells

### Manually set some macros

In [37]:
# get content and style images
# CONTENT_IDX: 1: Neckarfront    2: sjtu entrance
#              3: Shenzhen scene 4: Anime scene     5: anime
# STYLE_IDX:   1: starry night   2: a colorful artwork with multiple geometries
#              3: abstract art   4: Mondrian art
CONTENT_IDX = 1
STYLE_IDX = 1

WEIGHT_CONTENT = 1
WEIGHTS_STYLE = [1e11, 1e11, 1e11, 1e10, 1e10, 1e9, 1e9]
# WEIGHTS_STYLE = [1e6] * len(layers)

max_iter = 500
show_iter = 50

content_layers = [37]
style_layers = [9, 12, 16, 22, 25, 29, 32]

img_size = 512

### Define image-loading function

In [38]:
def load_images():
    img_dir = os.getcwd() + '/Images/'
    img_dirs = [img_dir, img_dir]
    img_names = ['content{}.png'.format(CONTENT_IDX), 'style{}.png'.format(STYLE_IDX)]
    imgs = [Image.open(img_dirs[i] + name) for i,name in enumerate(img_names)]
    return imgs

### Define pre-processing & post-processing transformations

In [39]:
prep = transforms.Compose([transforms.Scale(img_size),
                           transforms.ToTensor(),
                           transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]), #turn to BGR
                           transforms.Normalize(mean=[0.40760392, 0.45795686, 0.48501961],
                                                std=[1,1,1]),
                           transforms.Lambda(lambda x: x.mul_(255)),
                          ])
recover = transforms.Compose([transforms.Lambda(lambda x: x.mul_(1./255)),
                           transforms.Normalize(mean=[-0.40760392, -0.45795686, -0.48501961],
                                                std=[1,1,1]),
                           transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]), #turn to RGB
                           ])
toPIL = transforms.Compose([transforms.ToPILImage()])

def post(tensor): 
    tensor = recover(tensor)
    tensor[tensor > 1] = 1    
    tensor[tensor < 0] = 0
    tensor = toPIL(tensor)
    return tensor

### Image preparation

In [40]:
imgs = load_images()
imgs = [prep(img) for img in imgs]
imgs = [Variable(img.unsqueeze(0).to(device)) for img in imgs]

img_con, img_sty = imgs
opt_img = Variable(img_con.data.clone(), requires_grad=True)